In [2]:
from Paper_Results import TVSSegmentation #as pr
import Segmantation_lib as sg
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))
clstAlgo=['KMeans','specteral','affinity']
#kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
kc=[[9,6,6,4],[7,5,4,3],[10,7,9,6]]#Game of thrones
for s in range(1,3):
    #if s==1:
       # ep=8
    #else:
        #ep=14
    for e in range(1,11):
        print('season:',s,'episode:',e)
        try:
            df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
            trSegem=TVSSegmentation(tvs="g",season=s,episode=e)
            framesFeatures, timeStamp,shot=trSegem.load_Data()
            manScene_boundry=trSegem.load_ManAnnotation()
            _,manShot_boundry=sg.getShots(shot)
            manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
            manScene_boundry=[i/1000 for i in manScene_boundry]
            avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
            randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
            #uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

            avgFeatures=np.array(avgFeatures)
            randSampleFeatures=np.array(randSampleFeatures)
            randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
            uniSmplFeatures=np.array(uniSmplFeatures)
            uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

            shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
            shot_TxtEmb=np.array(shot_TxtEmb)
            shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

            conctAvgFeat=np.concatenate((avgFeatures,shot_TxtEmb),axis=1)
            conctRandFeat=np.concatenate((randSampleFeatures,shot_TxtEmb),axis=1)
            conctUniFeat=np.concatenate((uniSmplFeatures,shot_TxtEmb),axis=1)
            #print(conctAvgFeat.shape, conctRandFeat.shape,conctUniFeat.shape)

            manShot_boundry1=[i/1000 for i in manShot_boundry]
            shottimenp=np.array(manShot_boundry1[0:len(conctAvgFeat)])
            shottimenp=shottimenp/np.linalg.norm(shottimenp)
            conctAvgFeat_Time=np.column_stack((conctAvgFeat,shottimenp))
            conctRandFeat_Time=np.column_stack((conctRandFeat,shottimenp))
            conctUniFeat_Time=np.column_stack((conctUniFeat,shottimenp))
            conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

            dist_AvgFeat=trSegem.getSimilarityMatrix(conctAvgFeat,'cosine')
            dist_RandFeat=trSegem.getSimilarityMatrix(conctRandFeat,'cosine')
            dist_UniFeat=trSegem.getSimilarityMatrix(conctUniFeat,'cosine')
            dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
            dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
            dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')

            for i, clAl in enumerate(clstAlgo):
                avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
                randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
                uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
                avgFeatClusters=avgFeatClusters.tolist()
                randFeatClusters=randFeatClusters.tolist()
                uniFeatClusters=uniFeatClusters.tolist()
                avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
                randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
                uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
                avgFeatClusters_Time=avgFeatClusters_Time.tolist()
                randFeatClusters_Time=randFeatClusters_Time.tolist()
                uniFeatClusters_Time=uniFeatClusters_Time.tolist()

                sepPos=sg.sequenceSegmentation(avgFeatClusters,kc[i][0],kc[i][1])
                sepPos1=sg.sequenceSegmentation(randFeatClusters,kc[i][0],kc[i][1])
                sepPos2=sg.sequenceSegmentation(uniFeatClusters,kc[i][0],kc[i][1])
                sceneTime=[]
                sceneTime1=[]
                sceneTime2=[]
                sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,kc[i][2],kc[i][3])
                sceneTime_Time=[]
                sceneTime1_Time=[]
                sceneTime2_Time=[]
                for i in sepPos:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1:
                    sceneTime1.append(manShot_boundry[i])
                for i in sepPos2:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos2_Time:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos_Time:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1_Time:
                    sceneTime1.append(manShot_boundry[i])
                print('Auto',clAl,':',len(sepPos),len(sepPos1),len(sepPos_Time),len(sepPos1_Time),'the Manual:',len(manScene_boundry))
                tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
                len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)

                tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
                tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
                tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
                tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
                tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
                tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)

                truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
                truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,tAuto_Time2,tAuto_bin_old_Time2,tAuto_bin_old_Time2]
                truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]

                for i in range(len(truth_valueList)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df = df.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])

                for i in range(len(truth_valueList_Time)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df1 = df1.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])
        except:
            continue

season: 1 episode: 1
You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode01.en.srt
Auto KMeans : 34 31 81 67 the Manual: 44
Auto specteral : 32 38 74 90 the Manual: 44
Auto affinity : 34 39 47 49 the Manual: 44
season: 1 episode: 2
You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 2
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode02.en.srt
Auto KMeans : 25 23 53 43 the Manual: 29
Auto specteral : 25 25 64 59 the Manual: 29
Auto affinity : 25 30 30 37 the Manual: 29
season: 1 episode: 3
You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 3
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode03.en.srt
Auto KMeans : 34 32 65 60 the Manual: 30
Auto specteral : 29 29 78 81 the Manual: 30
Auto affinity : 31 34 38 41 the Manual: 30
season: 1 episode: 4
Y

In [3]:
df.to_csv('ResultsPaper/GoT/Results_segmentaion_TextEmbe_Augemnted_Samp_training.csv')
df1.to_csv('ResultsPaper/GoT/Results_segmentaion_TimeAugmented_Samp_training.csv')
dfBest.to_csv('ResultsPaper/GoT/Results_segmentaion_Features_Samp_training.csv')

In [6]:
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))
clstAlgo=['KMeans','specteral','affinity']
kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
#kc=[[9,6,6,4],[7,5,4,3],[10,7,9,6]]#Game of thrones
for s in range(3,4):
    #if s==1:
        #ep=8
    #else:
      #  ep=14
    for e in range(1,14):
        print('season:',s,'episode:',e)
        try:
            df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
            trSegem=TVSSegmentation(tvs="g",season=s,episode=e)
            framesFeatures, timeStamp,shot=trSegem.load_Data()
            manScene_boundry=trSegem.load_ManAnnotation()
            _,manShot_boundry=sg.getShots(shot)
            manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
            manScene_boundry=[i/1000 for i in manScene_boundry]
            avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
            randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
            #uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

            avgFeatures=np.array(avgFeatures)
            randSampleFeatures=np.array(randSampleFeatures)
            randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
            uniSmplFeatures=np.array(uniSmplFeatures)
            uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

            shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
            shot_TxtEmb=np.array(shot_TxtEmb)
            shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

            conctAvgFeat=np.concatenate((avgFeatures,shot_TxtEmb),axis=1)
            conctRandFeat=np.concatenate((randSampleFeatures,shot_TxtEmb),axis=1)
            conctUniFeat=np.concatenate((uniSmplFeatures,shot_TxtEmb),axis=1)
            #print(conctAvgFeat.shape, conctRandFeat.shape,conctUniFeat.shape)

            manShot_boundry1=[i/1000 for i in manShot_boundry]
            shottimenp=np.array(manShot_boundry1[0:len(conctAvgFeat)])
            shottimenp=shottimenp/np.linalg.norm(shottimenp)
            conctAvgFeat_Time=np.column_stack((conctAvgFeat,shottimenp))
            conctRandFeat_Time=np.column_stack((conctRandFeat,shottimenp))
            conctUniFeat_Time=np.column_stack((conctUniFeat,shottimenp))
            conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

            dist_AvgFeat=trSegem.getSimilarityMatrix(conctAvgFeat,'cosine')
            dist_RandFeat=trSegem.getSimilarityMatrix(conctRandFeat,'cosine')
            dist_UniFeat=trSegem.getSimilarityMatrix(conctUniFeat,'cosine')
            dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
            dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
            dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')

            for i, clAl in enumerate(clstAlgo):
                avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
                randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
                uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
                avgFeatClusters=avgFeatClusters.tolist()
                randFeatClusters=randFeatClusters.tolist()
                uniFeatClusters=uniFeatClusters.tolist()
                avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
                randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
                uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
                avgFeatClusters_Time=avgFeatClusters_Time.tolist()
                randFeatClusters_Time=randFeatClusters_Time.tolist()
                uniFeatClusters_Time=uniFeatClusters_Time.tolist()

                sepPos=sg.sequenceSegmentation(avgFeatClusters,kc[i][0],kc[i][1])
                sepPos1=sg.sequenceSegmentation(randFeatClusters,kc[i][0],kc[i][1])
                sepPos2=sg.sequenceSegmentation(uniFeatClusters,kc[i][0],kc[i][1])
                sceneTime=[]
                sceneTime1=[]
                sceneTime2=[]
                sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,kc[i][2],kc[i][3])
                sceneTime_Time=[]
                sceneTime1_Time=[]
                sceneTime2_Time=[]
                for i in sepPos:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1:
                    sceneTime1.append(manShot_boundry[i])
                for i in sepPos2:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos2_Time:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos_Time:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1_Time:
                    sceneTime1.append(manShot_boundry[i])
                print('Auto',clAl,':',len(sepPos),len(sepPos1),len(sepPos_Time),len(sepPos1_Time),'the Manual:',len(manScene_boundry))
                tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
                len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)

                tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
                tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
                tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
                tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
                tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
                tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)

                truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
                truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,tAuto_Time2,tAuto_bin_old_Time2,tAuto_bin_old_Time2]
                truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]

                for i in range(len(truth_valueList)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df = df.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])

                for i in range(len(truth_valueList_Time)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df1 = df1.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])
        except:
            continue

season: 3 episode: 1
You have chosen :
Tv-Series: Game of Thrones
Season:  3
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season03.Episode01.en.srt
Auto KMeans : 35 28 49 45 the Manual: 22
Auto specteral : 32 33 58 65 the Manual: 22
Auto affinity : 26 31 49 53 the Manual: 22
season: 3 episode: 2
You have chosen :
Tv-Series: Game of Thrones
Season:  3
Episode: 2
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season03.Episode02.en.srt
Auto KMeans : 36 30 44 40 the Manual: 23
Auto specteral : 29 31 62 60 the Manual: 23
Auto affinity : 25 28 39 46 the Manual: 23
season: 3 episode: 3
You have chosen :
Tv-Series: Game of Thrones
Season:  3
Episode: 3
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season03.Episode03.en.srt
season: 3 episode: 4
You have chosen :
Tv-Series: Game of Thrones
Season:  3
Episode: 4
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/

In [7]:
df.to_csv('ResultsPaper/BB/Results_segmentaion_TextEmbe_Augemnted_Samp_test_New.csv')
df1.to_csv('ResultsPaper/BB/Results_segmentaion_TimeAugmented_Samp_test_New.csv')
dfBest.to_csv('ResultsPaper/BB/Results_segmentaion_Features_Samp_test_New_Best.csv')

In [10]:
df_tmp

,winDiff,pk,coverage,purity,recall,precision,episode,season


In [11]:
dfBest

,winDiff,pk,coverage,purity,recall,precision,season,episode
KMeans,0.089,0.051,0.716,0.890,0.970,0.978,4,1
KMeans,0.089,0.047,0.696,0.851,0.974,0.979,4,1
KMeans,0.077,0.040,0.732,0.799,0.978,0.981,4,1
KMeans,0.139,0.078,0.562,0.945,0.942,0.978,4,1
KMeans,0.130,0.073,0.570,0.931,0.947,0.977,4,1
KMeans,1.857,0.934,0.599,0.926,0.048,0.938,4,1
specteral,0.089,0.047,0.724,0.808,0.974,0.979,4,1
specteral,0.087,0.047,0.702,0.873,0.971,0.981,4,1
specteral,0.081,0.041,0.769,0.798,0.978,0.980,4,1
specteral,0.147,0.081,0.541,0.963,0.937,0.980,4,1


In [ ]:
trSegem.fileName

In [4]:
trSegem=TVSSegmentation(tvs="g",season=1,episode=11)

You have chosen :
Tv-Series: Game of Thrones
Season:  1
Episode: 11
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season01.Episode11.en.srt


In [12]:
shottimenp=shottimenp/np.linalg.norm(shottimenp)

In [13]:
shottimenp

1.0

In [14]:
 _,manShot_boundry=sg.getShots(shot)
manShot_boundry1=[i/1000 for i in manShot_boundry]
shottimenp=np.array(manShot_boundry[0:len(conctAvgFeat)])
shottimenp=shottimenp/np.linalg.norm(shottimenp)

In [15]:
shottimenp

array([0.0068 , 0.0094 , 0.01116, 0.01624, 0.01648, 0.01692, 0.0172 ,
       0.0196 , 0.02088, 0.03872, 0.04   , 0.04164, 0.0488 , 0.05236,
       0.05256, 0.05336, 0.05356, 0.05648, 0.05672, 0.057  , 0.07688,
       0.0956 , 0.10332, 0.1084 , 0.10852, 0.1086 , 0.10876, 0.10892,
       0.10908, 0.1094 , 0.11048, 0.11104, 0.11964, 0.12476, 0.13648,
       0.138  , 0.14064, 0.14292, 0.14436, 0.14656, 0.15292, 0.15728,
       0.16952, 0.17112, 0.17484, 0.17648, 0.17808, 0.18388, 0.18564,
       0.19048, 0.19396, 0.19716, 0.20324, 0.2064 , 0.22   , 0.22592,
       0.23104, 0.23808, 0.24128, 0.24384, 0.24504, 0.24712, 0.24936,
       0.2578 , 0.2618 , 0.28508, 0.29412, 0.29792, 0.3058 , 0.30944,
       0.31288, 0.31488, 0.31768, 0.31984, 0.33336, 0.33528, 0.33984,
       0.34104, 0.34884, 0.36276, 0.365  , 0.36796, 0.37228, 0.37496,
       0.3762 , 0.37788, 0.38796, 0.39944, 0.4038 , 0.404  , 0.41268,
       0.41556, 0.42988, 0.4386 , 0.4444 , 0.44752, 0.45132, 0.454  ,
       0.4562 , 0.45

In [16]:
shottimenp=shottimenp/np.linalg.norm(shottimenp)
shottimenp

array([0.00010072, 0.00013923, 0.00016529, 0.00024053, 0.00024409,
       0.00025061, 0.00025475, 0.0002903 , 0.00030926, 0.00057349,
       0.00059245, 0.00061674, 0.00072279, 0.00077551, 0.00077848,
       0.00079032, 0.00079329, 0.00083653, 0.00084009, 0.00084424,
       0.00113868, 0.00141595, 0.00153029, 0.00160553, 0.00160731,
       0.00160849, 0.00161086, 0.00161323, 0.0016156 , 0.00162034,
       0.00163634, 0.00164463, 0.00177201, 0.00184784, 0.00202143,
       0.00204394, 0.00208304, 0.00211681, 0.00213814, 0.00217073,
       0.00226492, 0.0023295 , 0.00251079, 0.00253449, 0.00258959,
       0.00261388, 0.00263757, 0.00272348, 0.00274955, 0.00282123,
       0.00287277, 0.00292017, 0.00301022, 0.00305703, 0.00325846,
       0.00334614, 0.00342197, 0.00352624, 0.00357364, 0.00361156,
       0.00362933, 0.00366014, 0.00369331, 0.00381832, 0.00387756,
       0.00422237, 0.00435626, 0.00441254, 0.00452926, 0.00458317,
       0.00463412, 0.00466374, 0.00470521, 0.0047372 , 0.00493